# LlamaParser


- 링크: https://cloud.llamaindex.ai

In [ ]:
# 설치
# !pip install llama-index-core llama-parse llama-index-readers-file python-dotenv

In [20]:
import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

기본 파서 적용

In [49]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# 파서 설정
parser = LlamaParse(
    result_type="markdown",  # "markdown"과 "text" 사용 가능
    num_workers=8,  # worker 수 (기본값: 4)
    verbose=True,
    language="ko",
)

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}

# LlamaParse로 파일 파싱
documents = SimpleDirectoryReader(
    input_files=["data/SPRI_AI_Brief_2023년12월호_F.pdf"],
    file_extractor=file_extractor,
).load_data()

Started parsing the file under job_id 6a2aa79c-0d8b-4d59-866d-afa368baa31d


In [50]:
# 페이지 수 확인
len(documents)

23

LlamaIndex -> LangChain Document 로 변환

In [51]:
# 랭체인 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in documents]

In [52]:
# metadata 출력
docs[0].metadata

{'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_name': 'SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_type': 'application/pdf',
 'file_size': 975735,
 'creation_date': '2024-03-11',
 'last_modified_date': '2024-03-11'}

## MultiModal Model 로 파싱

**주요 파라미터**

- `use_vendor_multimodal_model`: 멀티모달 모델 사용 여부를 지정합니다. `True`로 설정하면 외부 벤더의 멀티모달 모델을 사용합니다.

- `vendor_multimodal_model_name`: 사용할 멀티모달 모델의 이름을 지정합니다. 여기서는 "openai-gpt4o"를 사용하고 있습니다.

- `vendor_multimodal_api_key`: 멀티모달 모델 API 키를 지정합니다. 환경 변수에서 OpenAI API 키를 가져옵니다.

- `result_type`: 파싱 결과의 형식을 지정합니다. "markdown"으로 설정되어 있어 결과가 마크다운 형식으로 반환됩니다.

- `language`: 파싱할 문서의 언어를 지정합니다. "ko"로 설정되어 한국어로 처리됩니다.

- `skip_diagonal_text`: 대각선 텍스트를 건너뛸지 여부를 결정합니다.

- `page_separator`: 페이지 구분자를 지정할 수 있습니다.

In [41]:
documents = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
    result_type="markdown",
    language="ko",
    # skip_diagonal_text=True,
    # page_separator="\n=================\n"
)

In [45]:
# parsing 된 결과
parsed_docs = documents.load_data(file_path="data/SPRI_AI_Brief_2023년12월호_F.pdf")

Started parsing the file under job_id cf2876e9-02c2-4277-ae92-03ae21d4a3bd


In [47]:
# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

아래와 같이 사용자 정의 인스트럭션을 지정하는 것도 가능합니다.

In [54]:
# parsing instruction 을 지정합니다.
parsing_instruction = (
    "You are parsing a brief of AI Report. Please extract tables in markdown format."
)

# LlamaParse 설정
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
    result_type="markdown",
    language="ko",
    parsing_instruction=parsing_instruction,
)

# parsing 된 결과
parsed_docs = parser.load_data(file_path="data/SPRI_AI_Brief_2023년12월호_F.pdf")

# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

Started parsing the file under job_id afdbf3ba-61f6-4c14-8d41-9e986950b612
.

In [56]:
# markdown 형식으로 추출된 테이블 확인
print(docs[-2].page_content)

# Ⅱ. 주요 행사 일정

| 행사명 | 행사 주요 개요 |
| --- | --- |
| CES 2024 | - 미국 소비자기술 협회(CTA)가 주관하는 세계 최대 가전·IT·소비재 전시회로 5G, AR&VR, 디지털헬스, 교통·모빌리티 등 주요 카테고리 중심으로 기업들이 최신의 기술 제품들을 전시<br>- CTA 사피로 회장은 가장 주목받는 섹터로 AI를 꼽았으며, 모든 산업을 포함한다는 의미에서 '올 인AI on'을 주제로 한 이번 전시에는 500곳 이상의 한국기업 참가 예정<br><br>![CES 2024](https://www.ces.tech/) |
| 기간 | 2024.1.9~12 |
| 장소 | 미국, 라스베가스 |
| 홈페이지 | [https://www.ces.tech/](https://www.ces.tech/) |

| 행사명 | 행사 주요 개요 |
| --- | --- |
| AIMLA 2024 | - 머신러닝 및 응용에 관한 국제 컨퍼런스(AIMLA 2024)는 인공지능 및 머신러닝의 이론, 방법론 및 실용적 접근에 관한 지식과 최신 연구 결과 공유<br>- 이론 및 실무 측면에서 인공지능, 기계학습의 주요 분야를 논의하고, 함께, 산업계의 연구자와 실무자들에게 해당 분야의 최첨단 개발 소식 공유<br><br>![AIMLA 2024](https://ccnet2024.org/aimla/index) |
| 기간 | 2024.1.27~28 |
| 장소 | 덴마크, 코펜하겐 |
| 홈페이지 | [https://ccnet2024.org/aimla/index](https://ccnet2024.org/aimla/index) |

| 행사명 | 행사 주요 개요 |
| --- | --- |
| AAAI Conference on Artificial Intelligence | - AI 발전 협회 컨퍼런스(AAAI)는 AI 연구를 촉진하고, AI 분야 연구원, 실무자, 과학자, 학술 및 공학자 간 교류의 기회 제공<br>- 컨퍼런스에서 AI 관련 기술 발표, 특별